In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv("/content/FA-KES-Dataset.csv",encoding='latin1')

In [ ]:
data.shape

(804, 7)

In [ ]:
data.head()

,unit_id,article_title,article_content,source,date,location,labels
0,1914947530,Syria attack symptoms consistent with nerve ag...,Wed 05 Apr 2017 Syria attack symptoms consiste...,nna,4/5/2017,idlib,0
1,1914947532,Homs governor says U.S. attack caused deaths b...,Fri 07 Apr 2017 at 0914 Homs governor says U.S...,nna,4/7/2017,homs,0
2,1914947533,Death toll from Aleppo bomb attack at least 112,Sun 16 Apr 2017 Death toll from Aleppo bomb at...,nna,4/16/2017,aleppo,0
3,1914947534,Aleppo bomb blast kills six Syrian state TV,Wed 19 Apr 2017 Aleppo bomb blast kills six Sy...,nna,4/19/2017,aleppo,0
4,1914947535,29 Syria Rebels Dead in Fighting for Key Alepp...,Sun 10 Jul 2016 29 Syria Rebels Dead in Fighti...,nna,7/10/2016,aleppo,0


In [ ]:
data.isnull().sum()

unit_id            0
article_title      0
article_content    0
source             0
date               0
location           0
labels             0
dtype: int64

In [ ]:
X = data.drop(columns=['labels'])
Y = data['labels']

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.5.0'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
# Vocabluary size
voc_size = 5000

In [ ]:
# ONEHOT Representation


In [ ]:
messages = X.copy()

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Dataset Preprocessing

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  #print(i)
  review = re.sub('[^a-zA-Z]',' ', messages['article_title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)


In [ ]:
corpus

['syria attack symptom consist nerv agent use',
 'hom governor say u attack caus death doesnt see big human loss',
 'death toll aleppo bomb attack least',
 'aleppo bomb blast kill six syrian state tv',
 'syria rebel dead fight key aleppo road',
 'suicid bomb kill least northeast syria',
 'dead heavi u raid syria stronghold',
 'suicid bomber kill assad clan hometown',
 'explos rock town damascu',
 'damascu explos due rocket bomb',
 'syrian regim step aerial assault douma',
 'hizballah lead regim offens southern syria',
 'syrian opposit remain divid',
 'video show murder syrian activist',
 'syria nusra front stage deadli suicid bomb aleppo',
 'regim troop thwart rebel attack syria aleppo',
 'ahrar al sham leader kill syria',
 'barrel bomb kill town syria',
 'rebel advanc north western syria',
 'isra strike syrian town kill pro regim fighter',
 'syria armi plane crash rebel held town',
 'syrian regim reveng attack kill score qalamoun',
 'chemic massacr idlib defi world',
 'bu bomb mark en

In [ ]:
onehot_repr = [one_hot(words,voc_size)for words in corpus]
onehot_repr

[[3812, 2266, 2507, 1027, 1852, 2984, 1470],
 [4571, 1733, 2628, 2223, 2266, 4337, 933, 3505, 2794, 4228, 4209, 311],
 [933, 3902, 1989, 1140, 2266, 2843],
 [1989, 1140, 3667, 1839, 1635, 1121, 3070, 4279],
 [3812, 460, 3823, 2233, 205, 1989, 3770],
 [3285, 1140, 1839, 2843, 1771, 3812],
 [3823, 798, 2223, 12, 3812, 3945],
 [3285, 2707, 1839, 3316, 868, 324],
 [3666, 1223, 4786, 350],
 [350, 3666, 880, 51, 1140],
 [1121, 3486, 163, 3195, 2008, 3655],
 [2371, 3126, 3486, 4919, 942, 3812],
 [1121, 4333, 1172, 3096],
 [1430, 727, 2177, 1121, 625],
 [3812, 3899, 4026, 1112, 3193, 3285, 1140, 1989],
 [3486, 4116, 2495, 460, 2266, 3812, 1989],
 [1261, 702, 2424, 4967, 1839, 3812],
 [604, 1140, 1839, 4786, 3812],
 [460, 3634, 4601, 3700, 3812],
 [1794, 1099, 1121, 4786, 1839, 1499, 3486, 2034],
 [3812, 844, 2349, 3863, 460, 2659, 4786],
 [1121, 3486, 3906, 2266, 1839, 688, 70],
 [264, 1742, 4809, 433, 2391],
 [3506, 1140, 3442, 3031, 1875, 4193, 4954, 3812],
 [1635, 1839, 1989, 1140, 4193, 11



**Embedding** **Representation**



In [ ]:
sent_length =20
embedded_docs = pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1852 2984 1470]
 [   0    0    0 ... 4228 4209  311]
 [   0    0    0 ... 1140 2266 2843]
 ...
 [   0    0    0 ... 1121 1989 4160]
 [   0    0    0 ... 1590 2542 3812]
 [   0    0    0 ... 1576 3797 3368]]


In [ ]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 3812, 2266, 2507, 1027, 1852, 2984, 1470], dtype=int32)

In [ ]:
# creating model
embedding_vector_features=40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(embedded_docs),Y.shape

(804, (804,))

In [ ]:
import numpy as np
X_final = np.array(embedded_docs)
Y_final = np.array(Y)

In [ ]:
X_final.shape , Y_final.shape

((804, 20), (804,))

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X_final,Y_final, test_size=0.33,random_state=42)

**Model** **Training**

In [ ]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
9/9 [==============================] - 3s 94ms/step - loss: 0.6930 - accuracy: 0.5112 - val_loss: 0.6904 - val_accuracy: 0.5564
Epoch 2/10
9/9 [==============================] - 0s 37ms/step - loss: 0.6912 - accuracy: 0.5167 - val_loss: 0.6887 - val_accuracy: 0.5564
Epoch 3/10
9/9 [==============================] - 0s 36ms/step - loss: 0.6873 - accuracy: 0.5688 - val_loss: 0.6914 - val_accuracy: 0.5376
Epoch 4/10
9/9 [==============================] - 0s 39ms/step - loss: 0.6800 - accuracy: 0.6859 - val_loss: 0.6907 - val_accuracy: 0.5301
Epoch 5/10
9/9 [==============================] - 0s 35ms/step - loss: 0.6626 - accuracy: 0.6413 - val_loss: 0.6957 - val_accuracy: 0.5075
Epoch 6/10
9/9 [==============================] - 0s 36ms/step - loss: 0.6229 - accuracy: 0.7286 - val_loss: 0.7300 - val_accuracy: 0.5000
Epoch 7/10
9/9 [==============================] - 0s 35ms/step - loss: 0.5655 - accuracy: 0.7100 - val_loss: 0.8114 - val_accuracy: 0.4850
Epoch 8/10
9/9 [===========

**Performance** **Metrics** **And** **Accuracy**

In [ ]:
Y_pred = model.predict_classes(x_test)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [ ]:
confusion_matrix(y_test,Y_pred)

array([[60, 58],
       [76, 72]])

In [ ]:
accuracy_score(y_test,Y_pred)

0.49624060150375937